# Reto datos abiertos Madrid

Análisis de los datos horarios de la calidad del aire descargados de datos.madrid.es

He descargado los datos del primer semestre de 2017.


In [67]:
#Importamos las librerías necesarias

import numpy as np
from os import listdir, path
import pandas as pd
import matplotlib.pyplot as plt


#### Creamos dos diccionarios con los datos de las estaciones y de los parámetros.


In [ ]:
estaciones = {"28079035":"Pza. del Carmen",
			  "28079004":"Pza. de España",
			  "28079039":"Barrio del Pilar",
			  "28079008":"Escuelas Aguirre",
			  "28079038":"Cuatro Caminos",
			  "28079011":"Av. Ramón y Cajal",
			  "28079040":"Vallecas",
			  "28079016":"Arturo Soria",
			  "28079017":"Villaverde Alto",
			  "28079018":"C/Farolillo",
			  "28079036":"Moratalaz",
			  "28079024":"Casa de Campo",
			  "28079027":"Barajas",
			  "28079047":"Méndez Álvaro",
			  "28079048":"Pº. Castellana",
			  "28079049":"Retiro",
			  "28079050":"Pza. Castilla",
			  "28079054":"Ensanche Vallecas",
			  "28079055":"Urb. Embajada(Barajas)",
			  "28079056":"Pza. Fdez. Ladreda",
			  "28079057":"Sanchinarro",
			  "28079058":"El Pardo",
			  "28079059":"Parque Juan Carlos I",
			  "28079060":"Tres Olivos"}

parametros = {"01":["Dióxido de Azufre", "SO2", "μg/m3"],
			  "06":["Monóxido de Carbono", "CO", "mg/m3"],
			  "07":["Monóxido de Nitrógeno", "NO", "μg/m3"],
			  "08":["Dióxido de Nitrógeno", "NO2", "μg/m3"],
			  "09":["Partículas < 2.5 μg", "PM2.5", "μg/m3"],
			  "10":["Partículas < 10 μg", "PM10", "μg/m3"],
			  "12":["Óxidos de Nitrógeno", "NOx", "μg/m3"],
			  "14":["Ozono", "O3", "μg/m3"],
			  "20":["Tolueno", "TOL","μg/m3"],
			  "30":["Benceno", "BEN", "μg/m3"],
			  "35":["Etilbenceno", "EBE", "μg/m3"],
			  "37":["Metalixeno", "MXY", "μg/m3"],
			  "38":["Paraxileno", "PXY", "μg/m3"],
			  "39":["Ortoxileno", "OXY", "μg/m3"],
			  "42":["Hidrocarburos totales (hexano)", "TCH", "mg/m3"],
			  "43":["Metano", "CH4", "mg/m3"],
			  "44":["Hidrocarburos no mecánicos (hexano)", "NMHC", "mg/m3"]}

DFparametros = pd.DataFrame(parametros)



#### Creamos la función obtenerDatos que obtiene un archivo pasado por parámetro y devuelve un dataframe con los datos del archivo. 

In [ ]:
#Funcion que lee los datos del archivo pasado por parámetro y los devuelve en una lista
def obtenerDatos(ruta, archivo):
  widths = [8,2,2,2,6,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1,5,1]
  tipos = ['String','String','String','String','String','Float','String','Float','String','Float','String','Float',
           'String','Float','String','Float','String','Float','String','Float','String','Float','String','Float',
           'String','Float','String','Float','String','Float','String','Float','String','Float','String','Float',
           'String','Float','String','Float','String','Float','String','Float','String','Float','String','Float',
           'String','Float','String','Float','String']
  return pd.read_fwf(path.join(ruta, archivo), widths=widths, header=None, types=tipos)#, names=names)



#### Obtenemos la lista de ficheros de la carpeta dat, recorremos la lista y llamamos a la función obtenerDatos por cada fichero. Juntamos los dataframes resultado de cada fichero en uno solo.

In [ ]:
ruta = 'dat/'
lista_ficheros = listdir(ruta)

#Llamamos a la funcion obtenerDatos por cada archivo de la carpeta
df = pd.DataFrame()
df = df.append([obtenerDatos(ruta,fichero) for fichero in lista_ficheros])

#Creamos una lista con los nombres de las columnas. Los campos "Hora" tienen el dato N o V en la columna siguiente
#que indica si es un dato válido o no. Se han nombrado las columnas de manera que hora_[par] es el dato y hora_[par+1] 
#es el indicador N o V.
names = ['cod_estacion', 'cod_parametro', 'cod_tecnica', 'cod_periodo',
         'fecha'] + ['hora_' + str(i) for i in range(48)]
#Asignamos los nombres a las columnas
df.columns = names


#### Obtenemos la media de los seis meses de cada hora por estación y parámetro

In [71]:

lista_pares = [0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46]
df_result = pd.DataFrame()
cont = 0
for i in lista_pares:
    serie = df.groupby(['cod_estacion', 'cod_parametro', 'hora_' + str(i+1)])['hora_' + str(i)].mean()
    df_result['column_'+ str(cont)] = serie
    cont = cont + 1 
      
#Parámetros que aparecen en el resultado
parametros_list = df_result.index.levels[0]
#Estaciones que aparecen en el resultado
estaciones_list = df_result.index.levels[1]

#print(df_result)

for i in parametros_list:
#    for j in estaciones_list:
    print(df_result.loc[i,:,'V'])

    


                                    column_0   column_1   column_2   column_3  \
cod_estacion cod_parametro hora_1                                               
28079004     1             V        5.370166   4.867403   4.461111   4.104972   
             6             V        0.456111   0.390000   0.342458   0.309444   
             7             V       37.403315  30.364641  23.627778  18.453039   
             8             V       50.265193  41.773481  35.455556  30.607735   

                                    column_4   column_5   column_6   column_7  \
cod_estacion cod_parametro hora_1                                               
28079004     1             V        3.872928   3.756906   4.281768   5.497238   
             6             V        0.291667   0.282222   0.328889   0.445556   
             7             V       14.828729  15.182320  26.022099  53.300000   
             8             V       26.552486  26.519337  33.784530  50.372222   

                          

                                    column_0   column_1   column_2   column_3  \
cod_estacion cod_parametro hora_1                                               
28079039     6             V        0.450562   0.367416   0.322034   0.298315   
             7             V       30.694444  18.333333  12.430168   9.611111   
             8             V       44.355556  34.988889  27.608939  24.138889   
             14            V       45.883333  49.255556  52.067039  52.205556   

                                    column_4   column_5   column_6   column_7  \
cod_estacion cod_parametro hora_1                                               
28079039     6             V        0.279775   0.274719   0.311864   0.419101   
             7             V        7.733333   7.400000  12.983240  33.222222   
             8             V       22.744444  22.816667  30.240223  47.405556   
             14            V       50.927778  49.505556  41.407821  29.044444   

                          